# BReact SDK Demo

This notebook demonstrates how to use the BReact SDK to interact with various services.

## Table of Contents
1. [Client Initialization](#Client-Initialization)
2. [Text Summarization Example](#Text-Summarization-Example)
3. [Email Generator Example](#Email-Generator-Example)
4. [Information Tracker](#Information-Tracker)
5. [Using Generic Services](#Using-Generic-Services)
6. [Cleanup](#Cleanup)


In [7]:
# Now import and use the client
from breact_sdk import BReactClient

# Initialize client
client = await BReactClient.create(
    api_key="00e0040a-02a9-4970-84f7-56f68b2b8046"
)

2025-02-03 12:02:56,311 - breact_sdk - INFO - Creating new BReactClient instance...
2025-02-03 12:02:56,311 - breact_sdk - INFO - Initializing BReactClient...
2025-02-03 12:02:56,312 - breact_sdk - ERROR - No API key provided


ValueError: API key must be provided either through BREACT_API_KEY environment variable or api_key parameter

## Text Summarization Example – Summarizing a Sample Text

In this section, we use the BReact SDK to generate a concise summary of a sample text. The parameters such as `summary_type`, `model_id`, and `options` control the summarization behavior.

In [4]:
sample_text = """
Artificial Intelligence (AI) has transformed various industries, from healthcare to finance.
Machine learning algorithms can now process vast amounts of data to identify patterns and make predictions.
Natural Language Processing has enabled computers to understand and generate human-like text.
Computer Vision systems can recognize objects and faces with remarkable accuracy.
These advancements have led to improved automation and decision-making capabilities.
"""

result = await client.ai_summarize(
    text=sample_text,
    summary_type="brief",
    model_id="mistral-small",
    options={
        "temperature": 0.3,
        "max_tokens": 500
    }
)

if result.status == "completed" and result.result:
    print("Summary:")
    print(result.result)
else:
    print(f"Error: {result.error or 'Unknown error'}")

2025-02-03 11:52:47,475 - breact_sdk - INFO - Summarizing text...
2025-02-03 11:52:47,476 - breact_sdk.http - INFO - Executing POST request to /api/v1/services/aisummary/summarize with polling
2025-02-03 11:52:47,476 - breact_sdk.http - INFO - Making POST request to /api/v1/services/aisummary/summarize
2025-02-03 11:52:47,510 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/15
2025-02-03 11:52:48,545 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/15
2025-02-03 11:52:49,584 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/15


Summary:
status='success' result=None error=None summary='Artificial Intelligence (AI) has significantly impacted multiple industries, including healthcare and finance, through enhanced data processing and pattern recognition. Key AI components, such as machine learning, natural language processing, and computer vision, have enabled improved automation and decision-making. Machine learning algorithms analyze large data sets to make predictions, while natural language processing allows computers to understand and generate human-like text. Computer vision systems excel in recognizing objects and faces with high precision.' metadata={'model': 'mistral-small', 'created_at': '2025-02-03T11:52:48.931144', 'original_length': 60, 'summary_length': 72, 'processing_time': '2025-02-03T11:52:48.931144', 'instance_id': 2621024405136, 'usage': {'prompt_tokens': 183, 'total_tokens': 277, 'completion_tokens': 94}}


## Email Generator Example – Analyzing and Responding to an Email Thread

This section demonstrates how to analyze an email thread for sentiment, key points, action items, and response urgency. Based on the analysis, a professional email response is generated using the BReact SDK.

In [6]:
# First, let's analyze the email thread
analysis_result = await client.analyze_email_thread(
    email_thread=[{
        "sender": "client@example.com",
        "recipient": "support@company.com",
        "subject": "Project Timeline Question",
        "content": """
        Hi Support Team,
        
        I'm wondering about the timeline for our current project. We seem to be running behind
        schedule on the database implementation phase. Could you provide an update on when
        we can expect this to be completed?
        
        Also, will this delay affect our final delivery date?
        
        Thanks,
        John
        """,
        "timestamp": "2024-03-20T10:00:00Z"
    }],
    analysis_type=["sentiment", "key_points", "action_items", "response_urgency"],
    model_id="mistral-small"
)

if analysis_result.status == "completed" and analysis_result.result:
    analysis = analysis_result.result.analysis  # Get the analysis dictionary
    print("Thread Analysis Results:")
    print(f"\nSentiment: {analysis['overall_sentiment']}")
    print("\nMain Points Discussed:")
    for point in analysis['main_points_discussed']:
        print(f"- {point}")
    print("\nAction Items:")
    for item in analysis['action_items']:
        print(f"- {item['task']} (Urgency: {item['urgency']})")
    print(f"\nResponse Urgency: {analysis['response_urgency']}")

    # Now generate a response based on the analysis
    response_result = await client.generate_email_response(
        email_thread=[{
            "sender": "client@example.com",
            "recipient": "support@company.com",
            "subject": "Project Timeline Question",
            "content": """
            Hi Support Team,
            
            I'm wondering about the timeline for our current project. We seem to be running behind
            schedule on the database implementation phase. Could you provide an update on when
            we can expect this to be completed?
            
            Also, will this delay affect our final delivery date?
            
            Thanks,
            John
            """,
            "timestamp": "2024-03-20T10:00:00Z"
        }],
        tone="professional",
        style_guide={
            "language": "en",
            "max_length": 200,
            "signature": "Best regards,\nSupport Team",
            "greeting_style": "formal"
        },
        key_points=[
            "Address timeline concerns",
            "Explain database implementation status",
            "Discuss impact on final delivery"
        ],
        model_id="mistral-small",
        options={
            "temperature": 0.3,
            "max_tokens": 1000
        }
    )
    
    if response_result.status == "completed" and response_result.result:
        response = response_result.result.response  # Get the response dictionary
        print("\nGenerated Response:")
        print(f"Subject: {response['subject']}")
        print("\nContent:")
        print(response['content'])
        
        if response['suggestions']:
            print("\nAlternative Suggestions:")
            for suggestion in response['suggestions']:
                print(f"- {suggestion}")
    else:
        print(f"\nError generating response: {response_result.error or 'Unknown error'}")
else:
    print(f"\nError analyzing thread: {analysis_result.error or 'Unknown error'}")

2025-02-03 11:52:49,645 - breact_sdk - INFO - Analyzing email thread...
2025-02-03 11:52:49,646 - breact_sdk.http - INFO - Executing POST request to /api/v1/services/email_response/analyze_thread with polling
2025-02-03 11:52:49,646 - breact_sdk.http - INFO - Making POST request to /api/v1/services/email_response/analyze_thread
2025-02-03 11:52:49,684 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/16
2025-02-03 11:52:50,725 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/16
2025-02-03 11:52:51,765 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/16
2025-02-03 11:52:52,815 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/16
2025-02-03 11:52:52,858 - breact_sdk - INFO - Generating email response...
2025-02-03 11:52:52,859 - breact_sdk.http - INFO - Executing POST request to /api/v1/services/email_response/generate_response with polling
2025-02-03 11:52:52,859 - breact_sdk.http - INFO - Maki

Thread Analysis Results:

Sentiment: neutral

Main Points Discussed:
- Project timeline question
- Concern about delay in database implementation phase
- Request for update on completion of database implementation phase
- Potential impact of delay on final delivery date

Action Items:
- Provide update on database implementation phase timeline (Urgency: high)
- Evaluate impact of delay on final delivery date (Urgency: high)

Response Urgency: high


2025-02-03 11:52:53,931 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/17
2025-02-03 11:52:54,971 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/17
2025-02-03 11:52:56,021 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/17
2025-02-03 11:52:57,075 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/17



Generated Response:
Subject: Re: Project Timeline Question

Content:
{
  "subject": "Response to Project Timeline Question",
  "content": "Dear John,\n\nI hope this message finds you well. I would be happy to provide an update regarding the timeline for our current project.\n\nWe understand that the database implementation phase is taking longer than initially anticipated. Our team is working diligently to resolve any issues and ensure a smooth and efficient implementation.\n\nAt this time, we expect the database implementation to be completed by 2024-03-27T10:00:00Z. Please note that this timeline is subject to change as we continue to work through any unforeseen challenges.\n\nRegarding the final delivery date, we anticipate a slight delay due to the extended database implementation phase. Our current estimate for final delivery is 2024-04-10T10:00:00Z. We will continue to monitor the situation and provide updates as necessary.\n\nThank you for your understanding and patience. If yo

## Information Tracker – Extracting Structured Information from Unstructured Text

Here we provide a JSON schema to extract structured medical information from a free-text note. The `extract_information` method uses the schema and additional context (`current_info`) to produce a structured analysis.

In [8]:
# Example of using the Information Tracker service
medical_note = """
Patient reports severe headache lasting for 3 days, accompanied by nausea and sensitivity to light. 
Pain level is described as 8/10. Symptoms worsen with physical activity. 
No prior history of migraines, but family history shows mother had similar symptoms.
"""

# Define the schema for medical symptom extraction
schema = {
    "type": "object",
    "properties": {
        "primary_symptom": {
            "type": "string",
            "enum": ["headache", "nausea", "dizziness", "fatigue", "pain"]
        },
        "duration": {
            "type": "string"
        },
        "pain_level": {
            "type": "string"
        },
        "associated_symptoms": {
            "type": "array",
            "items": {
                "type": "string"
            }
        },
        "aggravating_factors": {
            "type": "array",
            "items": {
                "type": "string"
            }
        },
        "family_history": {
            "type": "object",
            "properties": {
                "present": {
                    "type": "boolean"
                },
                "details": {
                    "type": "string"
                }
            },
            "required": ["present"]
        }
    },
    "required": ["primary_symptom", "duration", "pain_level"]
}

# Extract information using the new method
result = await client.extract_information(
    text=medical_note,
    schema=schema,
    update_type="medical_symptoms",
    current_info={
        "previous_symptoms": ["mild headache"]
    }
)

if result.status == "completed":
    if result.result.status == "error":
        print(f"Error extracting information: {result.result.error}")
    else:
        print("Analysis Results:")
        analysis = result.result.result
        
        print(f"\nPrimary Symptom: {analysis['primary_symptom']}")
        print(f"Duration: {analysis['duration']}")
        print(f"Pain Level: {analysis['pain_level']}")
        
        if 'associated_symptoms' in analysis:
            print("\nAssociated Symptoms:")
            for symptom in analysis['associated_symptoms']:
                print(f"- {symptom}")
        
        if 'aggravating_factors' in analysis:
            print("\nAggravating Factors:")
            for factor in analysis['aggravating_factors']:
                print(f"- {factor}")
        
        if 'family_history' in analysis:
            print("\nFamily History:")
            print(f"Present: {analysis['family_history']['present']}")
            if 'details' in analysis['family_history']:
                print(f"Details: {analysis['family_history']['details']}")
else:
    print(f"Error: {result.error or 'Unknown error'}")

2025-02-03 11:52:57,130 - breact_sdk.http - INFO - Executing POST request to /api/v1/services/information_tracker/extract_information with polling
2025-02-03 11:52:57,130 - breact_sdk.http - INFO - Making POST request to /api/v1/services/information_tracker/extract_information
2025-02-03 11:52:57,166 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/18
2025-02-03 11:52:58,348 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/18
2025-02-03 11:52:59,401 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/18
2025-02-03 11:53:00,438 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/18


Analysis Results:

Primary Symptom: headache
Duration: 3 days
Pain Level: 8/10

Associated Symptoms:
- nausea

Aggravating Factors:
- physical activity

Family History:
Present: True
Details: mother had similar symptoms


## Using Generic Services – Invoking Custom Service Endpoints

This section demonstrates how to call any available service endpoint using the `execute_service` method. This method allows flexibility to interact with services that may not have dedicated SDK implementations.

In [10]:
# Test webscraper service using the correct endpoint name
result = await client.execute_service(
    service_id="webscraper",
    endpoint="scrape",  # Changed from scrape_with_llm to scrape
    params={
        "url": "https://example.com",
        "extraction_description": "Extract the main article content",
        "model_id": "gpt-4o",  # optional
        "options": {  # optional
            "wait_until": "networkidle",
            "timeout": 30,
            "text_mode": True
        }
    }
)

print(f"Status: {result.status if hasattr(result, 'status') else 'N/A'}")
print("\nResult:")
print(result)

2025-02-03 11:53:00,495 - breact_sdk.http - INFO - Executing POST request to /api/v1/services/webscraper/scrape with polling
2025-02-03 11:53:00,496 - breact_sdk.http - INFO - Making POST request to /api/v1/services/webscraper/scrape
2025-02-03 11:53:00,672 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/19
2025-02-03 11:53:01,716 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/19
2025-02-03 11:53:03,758 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/19


Status: completed

Result:
status='completed' result=ServiceResult(status='success', result={'main_article_content': 'This domain is for use in illustrative examples in documents. You may use this domain in literature without prior coordination or asking for permission.'}, error=None, metadata={'url': 'https://example.com', 'created_at': '2025-02-03T11:53:00.671456', 'processing_time': 2.027872, 'model': 'gpt-4o'}) error=None message=None service='webscraper' endpoint='scrape' created_at=datetime.datetime(2025, 2, 3, 11, 53, 0, 661249) process_info=None


## Cleanup – Closing the Client

It is important to close the client after use to free up resources and properly terminate connections.

In [12]:
await client.close()
print("Client closed successfully!")

2025-02-03 11:53:03,807 - breact_sdk - INFO - Closing client and cleaning up resources...
2025-02-03 11:53:03,807 - breact_sdk.http - INFO - Closing HTTP client
2025-02-03 11:53:03,808 - breact_sdk - INFO - Client closed successfully


Client closed successfully!
